<a href="https://colab.research.google.com/github/Ayeshaaa777/Drug_recommender/blob/main/drug_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


In [39]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/drugscom_train.csv")

In [41]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [42]:
df=df.drop(columns=['Unnamed: 0'])

In [43]:
df=df.drop(columns=['date'])

In [44]:
df.head()

,drugName,condition,review,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37


In [45]:
pip install vaderSentiment

In [46]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [47]:
review_list = list(df.review)

In [48]:
sentiments = []
positive = []
negative = []
neutral = []
compound = []

In [49]:
for text in review_list:

  #Performing Vader Analysis on each review.
  com = analyser.polarity_scores(text)["compound"]
  pos = analyser.polarity_scores(text)["pos"]
  neu = analyser.polarity_scores(text)["neu"]
  neg = analyser.polarity_scores(text)["neg"]

  #Adding each value to the corresponding array
  positive.append(pos)
  negative.append(neg)
  neutral.append(neu)
  compound.append(com)
  sentiments.append({"Review":text,
                     "Positive": pos,
                     "Negative": neg,
                     "Neutral": neu,
                     "Compound": com})

sentiments_data = pd.DataFrame.from_dict(sentiments)

In [50]:
#Visualizing data from vader analysis.
sentiments_data

,Review,Positive,Negative,Neutral,Compound
0,"""It has no side effect, I take it in combinati...",0.000,0.121,0.879,-0.2960
1,"""My son is halfway through his fourth week of ...",0.108,0.018,0.874,0.9174
2,"""I used to take another oral contraceptive, wh...",0.080,0.059,0.861,0.6160
3,"""This is my first time using any form of birth...",0.089,0.026,0.885,0.7184
4,"""Suboxone has completely turned my life around...",0.168,0.061,0.771,0.9403
...,...,...,...,...,...
161292,"""I wrote my first report in Mid-October of 201...",0.143,0.037,0.820,0.9366
161293,"""I was given this in IV before surgey. I immed...",0.086,0.139,0.775,-0.4767
161294,"""Limited improvement after 4 months, developed...",0.135,0.462,0.404,-0.7430
161295,"""I&#039;ve been on thyroid medication 49 years...",0.113,0.066,0.821,0.8503


In [51]:
sentiments_data.shape

(161297, 5)

In [52]:
first_row_value = sentiments_data.loc[4, 'Review']
print(first_row_value)

"Suboxone has completely turned my life around.  I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account.  I had none of those before Suboxone and spent years abusing oxycontin.  My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction.  All that is history.  If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again.  I have found the side-effects to be minimal compared to oxycontin.  I&#039;m actually sleeping better.   Slight constipation is about it for me.  It truly is amazing. The cost pales in comparison to what I spent on oxycontin."


In [53]:
# Step 1: Find the maximum value in the "Positive" column
max_positive_value = sentiments_data['Positive'].max()

# Step 2: Filter the DataFrame to get the review with the highest positive value
review_with_max_positive = sentiments_data.loc[sentiments_data['Positive'] == max_positive_value]

# Step 3: Print the review with the highest positive value
print("Review with the highest positive value:")
print(review_with_max_positive)

Review with the highest positive value:
                         Review  Positive  Negative  Neutral  Compound
1090                    "Great"       1.0       0.0      0.0    0.6249
1112           "Great miracle."       1.0       0.0      0.0    0.8360
1914                  "Awesome"       1.0       0.0      0.0    0.6249
2193                     "Good"       1.0       0.0      0.0    0.4404
2452                 "Amazing."       1.0       0.0      0.0    0.5859
...                         ...       ...       ...      ...       ...
157315                   "Good"       1.0       0.0      0.0    0.4404
157344              "Excellent"       1.0       0.0      0.0    0.5719
158046  "Definitely effective."       1.0       0.0      0.0    0.7003
160281              "Brilliant"       1.0       0.0      0.0    0.5859
160920                   "Nice"       1.0       0.0      0.0    0.4215

[184 rows x 5 columns]


In [54]:
#Adding the sentiment analysis columns to the our dataset.
df["Positive"] = positive
df["Negative"] = negative
df["Neutral"] = neutral
df["Compound"] = compound
df["Review_Sentiment"] = ''


In [55]:
df.head()

,drugName,condition,review,rating,usefulCount,Positive,Negative,Neutral,Compound,Review_Sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27,0.000,0.121,0.879,-0.2960,
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192,0.108,0.018,0.874,0.9174,
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17,0.080,0.059,0.861,0.6160,
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10,0.089,0.026,0.885,0.7184,
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37,0.168,0.061,0.771,0.9403,


Based on the compound value we can determine whether the overall sentiment of the review is postive, negative or neutral. Below will be the threshold we shall be using for classifying the review sentiment class:

Positive sentiment: compound score >= 0.05

Neutral sentiment : -0.05 < compound score < 0.05

Negative sentiment: compound score <= -0.05

In [56]:
df.loc[df['Compound'] >= 0.05, 'Review_Sentiment'] = 'Positive'
df.loc[df['Compound'] <= -0.05, 'Review_Sentiment'] = 'Negative'
df["Review_Sentiment"].replace('', 'Neutral', inplace = True)


In [57]:

#Dropping the columns Positive, Negative, Neutral and Compound.
df = df.drop(columns = ["Positive", "Negative", "Neutral", "Compound"])


In [58]:
# Rearranging the columns in different order.
df = df[[ "drugName", "condition","review", "Review_Sentiment", "rating", "usefulCount"]]


In [59]:
df.head()

,drugName,condition,review,Review_Sentiment,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",Negative,9,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",Positive,8,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",Positive,5,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",Positive,8,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",Positive,9,37


In [60]:
#Exporting this merged dataset as csv file.
df.to_csv(f'/content/drive/MyDrive/Sentiment_analysis.csv', index=False)

**Weighted average of rating and useful count**


In [61]:
df.head()

,drugName,condition,review,Review_Sentiment,rating,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",Negative,9,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",Positive,8,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",Positive,5,17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",Positive,8,10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",Positive,9,37


In [62]:
#Sorting the data based on drug name
df = df.sort_values(['drugName'])

In [63]:

# Creating a function to calculate weighted average
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [64]:
df.groupby(["drugName"]).apply(wavg, "rating", "usefulCount")

drugName
A + D Cracked Skin Relief               10.000000
A / B Otic                              10.000000
Abacavir / dolutegravir / lamivudine     8.753846
Abacavir / lamivudine / zidovudine       9.000000
Abatacept                                8.234631
                                          ...    
Zyvox                                    9.293629
ZzzQuil                                  1.450000
depo-subQ provera 104                         NaN
ella                                     8.171134
femhrt                                   9.590909
Length: 3436, dtype: float64

In [65]:
# Creating a dataframe of Drug and its average rating
data_wavg = pd.DataFrame(df.groupby(["drugName"]).apply(wavg, "rating", "usefulCount").reset_index())


In [66]:
data_wavg.head()

,drugName,0
0,A + D Cracked Skin Relief,10.000000
1,A / B Otic,10.000000
2,Abacavir / dolutegravir / lamivudine,8.753846
3,Abacavir / lamivudine / zidovudine,9.000000
4,Abatacept,8.234631


In [67]:
data_wavg = data_wavg.rename(columns={0: "Rating_Wavg"})

In [68]:
data_wavg.head()

,drugName,Rating_Wavg
0,A + D Cracked Skin Relief,10.000000
1,A / B Otic,10.000000
2,Abacavir / dolutegravir / lamivudine,8.753846
3,Abacavir / lamivudine / zidovudine,9.000000
4,Abatacept,8.234631


In [69]:

merged_wavg = pd.merge(data_wavg, df, on='drugName')

In [70]:
merged_wavg.head()

,drugName,Rating_Wavg,condition,review,Review_Sentiment,rating,usefulCount
0,A + D Cracked Skin Relief,10.000000,Bacterial Skin Infection,"""I have severe cracked skin on my hands. I&#0...",Positive,10,6
1,A / B Otic,10.000000,Otitis Media,"""It numbs the pain. It makes my ear feel heavi...",Positive,10,20
2,Abacavir / dolutegravir / lamivudine,8.753846,HIV Infection,"""I was diagnosed in January 2011. My own immun...",Negative,9,6
3,Abacavir / dolutegravir / lamivudine,8.753846,HIV Infection,"""I recently found out about my positive status...",Positive,7,26
4,Abacavir / dolutegravir / lamivudine,8.753846,HIV Infection,"""I was diagnosed in 2007 and had since never h...",Neutral,10,9


In [71]:
merged_wavg.shape

(161297, 7)

**Drug Recommendation**

In [72]:
merged_wavg = merged_wavg.sort_values(['condition','Rating_Wavg'],ascending=False, ignore_index=True).groupby('condition').head(161297)
merged_wavg

,drugName,Rating_Wavg,condition,review,Review_Sentiment,rating,usefulCount
0,Voltaren,8.854352,zen Shoulde,"""Great help""",Positive,8,33
1,Indomethacin,8.757336,zen Shoulde,"""It works.""",Neutral,10,12
2,Diclofenac,8.617426,zen Shoulde,"""While taking this medication I had increased ...",Negative,9,13
3,Diclofenac,8.617426,zen Shoulde,"""This medication has been a God send for me. ...",Negative,9,11
4,Diclofenac,8.617426,zen Shoulde,"""Great help""",Positive,8,33
...,...,...,...,...,...,...,...
160393,Depo-Provera,4.875840,0</span> users found this comment helpful.,"""Depo worked great for me. Reading people&#039...",Positive,10,0
160394,Depo-Provera,4.875840,0</span> users found this comment helpful.,"""I have liked depo, but today I started bleedi...",Positive,9,0
160395,Depo-Provera,4.875840,0</span> users found this comment helpful.,"""This the only birth control that gets rid of ...",Positive,9,0
160396,Ogestrel-28,4.200000,0</span> users found this comment helpful.,"""I have been on many different birth control p...",Positive,10,0


In [73]:
# Taking drugs only with positive reviews for recommendation
merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Negative"].index)
merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Neutral"].index)


In [74]:
merged_wavg.head()


,drugName,Rating_Wavg,condition,review,Review_Sentiment,rating,usefulCount
0,Voltaren,8.854352,zen Shoulde,"""Great help""",Positive,8,33
4,Diclofenac,8.617426,zen Shoulde,"""Great help""",Positive,8,33
5,Relafen,8.442509,zen Shoulde,"""I&#039;m probably the only one I know taking ...",Positive,7,50
6,Ibuprofen,8.428997,zen Shoulde,"""I&#039;ve found that taking ibuprofen (200 mg...",Positive,8,0
8,Naproxen,7.947768,zen Shoulde,"""Very little relief. I finished PT and after ...",Positive,2,6


In [75]:
# taking predicted disease as input and recommending drug based on highest weighted average of ratings
groupedByCount = merged_wavg.groupby(['condition', 'drugName', 'Rating_Wavg'])['usefulCount'].sum().reset_index()

In [76]:
groupedByCount


,condition,drugName,Rating_Wavg,usefulCount
0,0</span> users found this comment helpful.,Aviane,7.550182,0
1,0</span> users found this comment helpful.,Bepreve,6.694915,0
2,0</span> users found this comment helpful.,Cryselle,6.963415,0
3,0</span> users found this comment helpful.,Depo-Provera,4.875840,0
4,0</span> users found this comment helpful.,Drysol,9.385661,0
...,...,...,...,...
6392,zen Shoulde,Ibuprofen,8.428997,0
6393,zen Shoulde,Nabumetone,7.513756,50
6394,zen Shoulde,Naproxen,7.947768,6
6395,zen Shoulde,Relafen,8.442509,50


In [77]:
groupedByCondition = groupedByCount.groupby('condition')

In [80]:
predicted_drug = pd.DataFrame(groupedByCondition.get_group('Otitis Media').nlargest(3, ['Rating_Wavg', 'usefulCount']))
predicted_drug

,condition,drugName,Rating_Wavg,usefulCount
4621,Otitis Media,Antipyrine / benzocaine,10.000000,34
4618,Otitis Media,A / B Otic,10.000000,20
4627,Otitis Media,Ceftriaxone,9.113786,17
